## 스크래핑
 - G마켓 베스트 셀러 페이지 클롤링
 - 크롤링한 데이터를 전처리(원가 데이터가 없는 경우 판매가를 넣음)
 - 할인율을 계산한 칼럼을 추가
 - 할인율이 50%이상인 상품만 필터링

In [7]:
import requests
from selenium import webdriver
import time

#웹 드라이버 실행 및 페이지 이동
driver = webdriver.Chrome('C:/myexam/chromedrive/chromedriver.exe')
url = 'http://corners.gmarket.co.kr/Bestsellers'
driver.get(url)

#아이템 엘레멘트를 선택
items = driver.find_elements_by_css_selector('.best-list:nth-child(3) >ul>li')
len(items)

200

In [8]:
#제목 링크, 원가, 판매가 텍스트 데이터를 크롤링
dict_list = []
for item in items:
    title = item.find_element_by_css_selector(".itemname").text
    link = item.find_element_by_css_selector(".itemname").get_attribute("href")
    o_price = item.find_element_by_css_selector(".o-price").text[:-1]
    s_price = item.find_element_by_css_selector(
        ".s-price").text.split(" ")[0][:-1]

    data = {
        "title": title,
        "link": link,
        "o_price": o_price,
        "s_price": s_price,
    }

    dict_list.append(data)

In [9]:
#브라우저 닫기
driver.quit()

In [11]:
# 데이터 프라임 만들기
import pandas as pd

df = pd.DataFrame(dict_list)
df.tail()

,title,link,o_price,s_price
195,[베베앙]베베앙 아기물티슈 시그니처 대용량 100매 캡형 10팩,http://item.gmarket.co.kr/Item?goodscode=16044...,"23,800","11,900"
196,[페리오]어린이칫솔 쿠폰가13530원 2단계핑크퐁칫솔20입+사은품,http://item.gmarket.co.kr/Item?goodscode=17038...,"20,100","17,900"
197,수퍼맘스토리카페/재밌고 기억하기 쉽게 가르쳐주는/미국초등수학교과서/맥그로힐/McGr...,http://item.gmarket.co.kr/Item?goodscode=21923...,"68,000","48,800"
198,[구글플레이](카드가능) 기프트코드 20만원 / 구글 기프트카드,http://item.gmarket.co.kr/Item?goodscode=16552...,"200,000","190,000"
199,[매일유업]매일두유 99.9 190ml 48팩,http://item.gmarket.co.kr/Item?goodscode=15900...,"28,900","22,500"


값이 없는 o_price를 s_price로 넣기

In [12]:
nodata_index = df['o_price'] == ""

In [13]:
df['o_price'][nodata_index].tail()

184    
188    
190    
191    
194    
Name: o_price, dtype: object

In [15]:
df['s_price'][nodata_index].tail()

184     56,500
188      5,990
190     30,900
191    149,000
194     19,900
Name: s_price, dtype: object

In [16]:
df[nodata_index].tail()

,title,link,o_price,s_price
184,윤쌀 10kg+10kg 쌀 20년산 (박스 포장),http://item.gmarket.co.kr/Item?goodscode=18505...,,"56,500"
188,(전단상품)온가족닭볶음탕용 1.2kg 팩,http://item.gmarket.co.kr/Item?goodscode=20446...,,"5,990"
190,비비고 만두 총 10팩(왕교자X2+김치왕교자X2+매운왕교자X2+고기만두X2+김치만,http://item.gmarket.co.kr/Item?goodscode=21251...,,"30,900"
191,[삼성전자]7%할인 갤럭시버즈2+케이스+CU8천원+경품EVENT,http://item.gmarket.co.kr/Item?goodscode=21898...,,"149,000"
194,향긋한 여름 털복숭아 4.5kg/20과내외,http://item.gmarket.co.kr/Item?goodscode=21607...,,"19,900"


In [18]:
# 값이 없는 o_price를 s_price로 넣기
df['o_price'][nodata_index] = df['s_price'][nodata_index]

In [19]:
df.loc[nodata_index].tail()

,title,link,o_price,s_price
184,윤쌀 10kg+10kg 쌀 20년산 (박스 포장),http://item.gmarket.co.kr/Item?goodscode=18505...,"56,500","56,500"
188,(전단상품)온가족닭볶음탕용 1.2kg 팩,http://item.gmarket.co.kr/Item?goodscode=20446...,"5,990","5,990"
190,비비고 만두 총 10팩(왕교자X2+김치왕교자X2+매운왕교자X2+고기만두X2+김치만,http://item.gmarket.co.kr/Item?goodscode=21251...,"30,900","30,900"
191,[삼성전자]7%할인 갤럭시버즈2+케이스+CU8천원+경품EVENT,http://item.gmarket.co.kr/Item?goodscode=21898...,"149,000","149,000"
194,향긋한 여름 털복숭아 4.5kg/20과내외,http://item.gmarket.co.kr/Item?goodscode=21607...,"19,900","19,900"


In [20]:
df.tail()

,title,link,o_price,s_price
195,[베베앙]베베앙 아기물티슈 시그니처 대용량 100매 캡형 10팩,http://item.gmarket.co.kr/Item?goodscode=16044...,"23,800","11,900"
196,[페리오]어린이칫솔 쿠폰가13530원 2단계핑크퐁칫솔20입+사은품,http://item.gmarket.co.kr/Item?goodscode=17038...,"20,100","17,900"
197,수퍼맘스토리카페/재밌고 기억하기 쉽게 가르쳐주는/미국초등수학교과서/맥그로힐/McGr...,http://item.gmarket.co.kr/Item?goodscode=21923...,"68,000","48,800"
198,[구글플레이](카드가능) 기프트코드 20만원 / 구글 기프트카드,http://item.gmarket.co.kr/Item?goodscode=16552...,"200,000","190,000"
199,[매일유업]매일두유 99.9 190ml 48팩,http://item.gmarket.co.kr/Item?goodscode=15900...,"28,900","22,500"


In [21]:
#할인률 계산해서 칼럼에 추가하기
df["o_price"] = df["o_price"].apply(
    lambda price: price.replace(",", "")).astype("int")
df["s_price"] = df["s_price"].apply(
    lambda price: price.replace(",", "")).astype("int")
df["sale_rate"] = round((1 - df["s_price"] / df["o_price"]) * 100, 2)
df.tail()

,title,link,o_price,s_price,sale_rate
195,[베베앙]베베앙 아기물티슈 시그니처 대용량 100매 캡형 10팩,http://item.gmarket.co.kr/Item?goodscode=16044...,23800,11900,50.00
196,[페리오]어린이칫솔 쿠폰가13530원 2단계핑크퐁칫솔20입+사은품,http://item.gmarket.co.kr/Item?goodscode=17038...,20100,17900,10.95
197,수퍼맘스토리카페/재밌고 기억하기 쉽게 가르쳐주는/미국초등수학교과서/맥그로힐/McGr...,http://item.gmarket.co.kr/Item?goodscode=21923...,68000,48800,28.24
198,[구글플레이](카드가능) 기프트코드 20만원 / 구글 기프트카드,http://item.gmarket.co.kr/Item?goodscode=16552...,200000,190000,5.00
199,[매일유업]매일두유 99.9 190ml 48팩,http://item.gmarket.co.kr/Item?goodscode=15900...,28900,22500,22.15


In [22]:
df[df["sale_rate"] > 50].sort_values("sale_rate", ascending=False).reset_index(drop=True).head()

,title,link,o_price,s_price,sale_rate
0,엄마옷로즈맘 시즌오프세일 인견5060중년여성의류,http://item.gmarket.co.kr/Item?goodscode=77658...,43000,12900,70.0
1,[엄지한복]20%+15% 엄지한복 남아/여아 균일가+복주머니 증정,http://item.gmarket.co.kr/Item?goodscode=16013...,130000,39000,70.0
2,[벨레다]벨레다공식총판 벨레다 어린이치약 50ml 6개 +증정품,http://item.gmarket.co.kr/Item?goodscode=19039...,143330,43000,70.0
3,데이데이 가을블라우스/맨투맨/팬츠/니트/가디건/팬츠,http://item.gmarket.co.kr/Item?goodscode=15706...,33000,9900,70.0
4,[캔디베이비]70%할인 시즌오프 유아동내의/내복/실내복,http://item.gmarket.co.kr/Item?goodscode=43858...,19000,5700,70.0
